In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def bart_summarize(text, max_len=130, min_len=30):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_len,
        min_length=min_len,
        length_penalty=2.0,
        early_stopping=True,
    )

    # Decode summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [4]:
from datasets import load_dataset

ds = load_dataset("cnn_dailymail", "3.0.0")
article = ds["train"][0]["article"]

summary = bart_summarize(article)
print(summary)


Harry Potter star Daniel Radcliffe turns 18 on Monday. He gains access to a reported £20 million ($41.1 million) fortune. Radcliffe's earnings from the first five Potter films have been held in a trust fund.


In [5]:
import evaluate

rouge = evaluate.load("rouge")


In [6]:
from datasets import load_dataset

# Load dataset
ds = load_dataset("cnn_dailymail", "3.0.0")

article = ds["train"][0]["article"]
reference = ds["train"][0]["highlights"]   # human summary

# Your abstractive summary
abstractive_summary = bart_summarize(article)


In [7]:
abstractive_scores = rouge.compute(
    predictions=[abstractive_summary],
    references=[reference]
)

abstractive_scores


{'rouge1': np.float64(0.6578947368421052),
 'rouge2': np.float64(0.43243243243243246),
 'rougeL': np.float64(0.631578947368421),
 'rougeLsum': np.float64(0.631578947368421)}